# 自动微分

自动微分是 PyTorch 的一个非常重要的功能，也是驱动深度学习快速发展的重要原因。

首先看一个简单的例子。考虑函数 $f(x,y)=x\cdot \log(x)+\sin(xy)$，我们想求 $f$ 在 $(x,y)=(1,2)$ 处对 $x$ 和 $y$ 的偏导。

首先创建 Tensor 对象：

In [2]:
import torch

x = torch.tensor([1.0])
y = torch.tensor([2.0])
x

tensor([1.])

为了告诉 PyTorch 要对 `x` 和 `y` 求导，我们需要设置 `requires_grad` 属性：

In [3]:
x.requires_grad = True
y.requires_grad = True
x

tensor([1.], requires_grad=True)

In [4]:
x = torch.tensor([1.0], requires_grad = True)
y = torch.tensor([2.0], requires_grad = True)
x

tensor([1.], requires_grad=True)

接下来利用 PyTorch 定义的运算计算函数值：

In [11]:
f = x * torch.log(x) + torch.sin(x * y)
f

tensor([0.9093], grad_fn=<AddBackward0>)

然后调用 `backward()` 函数进行反向传播：

In [12]:
f.backward()

此时 `x` 和 `y` 会有一个 `grad` 属性，即为计算出的导数值：

In [13]:
print(x.grad)
print(y.grad)

tensor([0.5198])
tensor([-1.2900])


我们可以手动计算 $\partial f/\partial x=\log(x)+1+y\cos(xy)$，$\partial f/\partial y=x\cos(xy)$，对结果进行验证。因为此时不再需要 PyTorch 记录导数，所以可以把结果放在 `torch.no_grad()` 中：

In [11]:
with torch.no_grad(): # with 会创建一个小的subenv，在其中都是no_grad()的
    print(torch.log(x) + 1.0 + y * torch.cos(x * y))
    print(x * torch.cos(x * y))

tensor([0.1677])
tensor([-0.4161])


自动微分同样适用于任意形状的 Tensor，包括向量和矩阵。例如 $f(x,y)=(x+y)'(x+y)$，其中 $x$ 和 $y$ 为向量。

In [12]:
torch.manual_seed(123)

x = torch.randn(5)
y = torch.rand(5)
x.requires_grad = True
y.requires_grad = True

f = (x + y).dot(x + y) 
f.backward()

print(x.grad)
print(y.grad)
print(2.0 * (x + y))

tensor([-0.0717,  0.6340, -0.1064,  0.3226, -2.1567])
tensor([-0.0717,  0.6340, -0.1064,  0.3226, -2.1567])
tensor([-0.0717,  0.6340, -0.1064,  0.3226, -2.1567], grad_fn=<MulBackward0>)


**思考题**：给定一个行列式为正的矩阵 $X$，定义 $f(X)=\log\det(X)$，其中 $\det(X)$ 为 $X$ 的行列式。那么 $\partial f/\partial X$ 应该是什么？

In [16]:
torch.manual_seed(123)

x = torch.randn(5, 5)
if x.det().item() < 0:
    x = -x

print(x.det().log())

x.requires_grad = True
f = torch.logdet(x)
f.backward()

print(x)
print(x.grad)

tensor(1.1183)
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486],
        [ 0.6603, -0.2196, -0.3792,  0.7671, -0.4015],
        [ 0.6957, -1.8061,  1.8960, -0.1750,  1.3689],
        [-1.6033, -1.3250, -1.4096, -0.4076,  0.7953],
        [ 0.9985,  0.2212,  1.8319, -0.3378,  0.8805]], requires_grad=True)
tensor([[ 0.7673, -0.2915, -0.4447, -1.1218, -0.3021],
        [ 0.5376,  0.2348, -0.5945,  1.1406,  1.0058],
        [ 0.0020, -0.6879,  0.3541, -0.4003, -0.7196],
        [-0.1782,  0.3465, -0.3755,  0.6708,  1.1536],
        [ 0.0992,  0.9788, -0.3062,  0.9806,  1.7905]])
